# Test: Async Agent with MCP Workbench in Jupyter (Linux)
This notebook tests that async agent code using MCP Workbench does not hang in Jupyter.

In [ ]:
# Install nest_asyncio if not already installed
!pip install nest_asyncio --quiet

In [ ]:
import nest_asyncio
nest_asyncio.apply()
print("nest_asyncio applied")

In [ ]:
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.mcp import McpWorkbench, StdioServerParams

# --- API Key Handling Improvement ---
# Check for API key and prompt user if not set
if not os.environ.get("OPENAI_API_KEY"):
    from getpass import getpass
    api_key = getpass("Enter your OpenAI API Key (input hidden): ")
    os.environ["OPENAI_API_KEY"] = api_key
    print("API key set for this session.")
else:
    print("OPENAI_API_KEY found in environment.")

fetch_mcp_server = StdioServerParams(command="uvx", args=["mcp-server-fetch"])

async def test_agent():
    async with McpWorkbench(fetch_mcp_server) as workbench:  # type: ignore
        model_client = OpenAIChatCompletionClient(model="gpt-4.1-nano")
        fetch_agent = AssistantAgent(
            name="fetcher", model_client=model_client, workbench=workbench, reflect_on_tool_use=True
        )
        result = await fetch_agent.run(task="Summarize the content of https://en.wikipedia.org/wiki/Seattle")
        assert isinstance(result.messages[-1], TextMessage)
        print(result.messages[-1].content)
        await model_client.close()

import asyncio
await test_agent()

**Note:** This notebook now securely prompts for your OpenAI API key if it is not set in the environment, so you do not need to edit the code to add your key. This improves usability and security. If you see an error about `OPENAI_API_KEY`, ensure you enter a valid key when prompted.